In [ ]:

df = pd.read_csv("./generated_data.csv")

df["GTU"] = df.TPS / 10

df['Date'] = df.TR_YMD_HMS.apply(lambda x: str(x)[:14])
df['year'] = df.TR_YMD_HMS.apply(lambda x: str(x)[:4])
df['month'] = df.TR_YMD_HMS.apply(lambda x: str(x)[4:6])
df['day'] = df.TR_YMD_HMS.apply(lambda x: str(x)[6:8])
df['hour'] = df.TR_YMD_HMS.apply(lambda x: str(x)[8:10])
df['mintue'] = df.TR_YMD_HMS.apply(lambda x: str(x)[10:12])


df.Date = pd.to_datetime(df.Date.astype(str), format="%Y%m%d%H%M%S")

df["dayofweek"] = df.Date.dt.dayofweek

In [ ]:
df.TR_YMD_HMS = df.TR_YMD_HMS.apply(lambda x: str(x)[:14])

In [ ]:
df.dropna()

In [ ]:
comp = comp.dropna()

In [ ]:
df

In [ ]:
tr_1 = df[(df.Date.dt.month<= 8)]
tr_1 = tr_1[['TPS', 'GTU', 'ELPS_AVG', 'ELPS_MIN', 'ELPS_MAX', 'Date', 'year', 'month', 'day', 'hour', 'mintue', 'dayofweek']]

tr_1

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb


df = pd.read_csv("./generated_data.csv")
df=df.dropna()
df["GTU"] = df.TPS / 10

df['Date'] = df.TR_YMD_HMS.apply(lambda x: str(x)[:14])
df['year'] = df.TR_YMD_HMS.apply(lambda x: str(x)[:4])
df['month'] = df.TR_YMD_HMS.apply(lambda x: str(x)[4:6])
df['day'] = df.TR_YMD_HMS.apply(lambda x: str(x)[6:8])
df['hour'] = df.TR_YMD_HMS.apply(lambda x: str(x)[8:10])
df['mintue'] = df.TR_YMD_HMS.apply(lambda x: str(x)[10:12])

df.Date = pd.to_datetime(df.Date.astype(str), format="%Y%m%d%H%M%S")

df["dayofweek"] = df.Date.dt.dayofweek

comp = pd.read_csv("./shooting_log (1).csv")

comp['Date'] = comp.TR_YMD_HMS.apply(lambda x: str(x)[:14])
comp['year'] = comp.TR_YMD_HMS.apply(lambda x: str(x)[:4])
comp['month'] = comp.TR_YMD_HMS.apply(lambda x: str(x)[4:6])
comp['day'] = comp.TR_YMD_HMS.apply(lambda x: str(x)[6:8])
comp['hour'] = comp.TR_YMD_HMS.apply(lambda x: str(x)[8:10])
comp['mintue'] = comp.TR_YMD_HMS.apply(lambda x: str(x)[10:12])


comp.Date = pd.to_datetime(comp.Date.astype(str), format="%Y%m%d%H%M%S")

comp["dayofweek"] = comp.Date.dt.dayofweek

#----------------------------------------GTU----------------------------------------------------------------------------------

def GTU(tr_1, new_df, vali, test):
    x = tr_1[["day", "hour", "mintue", "dayofweek"]]   
    print(x.dtypes)
    #train데이터 사용
    y = tr_1[["GTU"]]
    
    xvali = vali[["day", "hour", "mintue", "dayofweek"]]        #validation 사용
    yvali = vali[["GTU"]]

    eval_set = [(xvali, yvali)]

    new_df["Date"] = test["Date"]

    test = test[["day", "hour", "mintue", "dayofweek"]]            #예측데이터

    new_df['day'] = test['day']
    new_df['hour'] = test['hour']
    new_df['mintue'] = test['mintue']
    new_df['dayofweek'] = test['dayofweek']
    new_df.reset_index()

    xgb_model = xgb.XGBRegressor(n_estimators= 250, learning_rate=0.07, subsample=0.7,
                        colsample_bytree=0.7, max_depth=6, silent = 1, nthread = 4, min_child_weight = 4,enable_categorical=True)

    
    xgb_model.fit(x, y, eval_set=eval_set, eval_metric="rmse", early_stopping_rounds=15)
    pred = xgb_model.predict(test)

    new_df['GTU'] = pred

    return new_df

#---------------------------------ELPS_MSEC_SUM----------------------------------------------------------------------------------
def ELPS_SUM(tr_1, new_df, vali):
    x = tr_1[["day", "hour", "mintue", "dayofweek", "GTU"]]
    y = tr_1[["ELPS_AVG"]]

    xvali = vali[["day", "hour", "mintue", "dayofweek", "GTU"]]        #validation 사용
    yvali = vali[["ELPS_AVG"]]

    eval_set = [(xvali, yvali)]

    test = new_df[["day", "hour", "mintue", "dayofweek", "GTU"]]            #예측데이터

    xgb_model = xgb.XGBRegressor(n_estimators=250, learning_rate=0.07, subsample=0.7,
                        colsample_bytree=0.7, max_depth=6, silent = 1, nthread = 4, min_child_weight = 4)

    
    xgb_model.fit(x, y, eval_set= eval_set, eval_metric="rmse", early_stopping_rounds=15,enable_categorical=True)
    pred = xgb_model.predict(test)

    new_df['ELPS_AVG'] = pred

    return new_df

#-----------------------------------------------------ELPS_MSEC_MIN-------------------------------------------------------------------------

def ELPS_MIN(tr_1, new_df, vali):

    x = tr_1[["day", "hour", "mintue", "dayofweek", "GTU"]]
    x.astype(str)
    y = tr_1[["ELPS_MIN"]]

    xvali = vali[["day", "hour", "mintue", "dayofweek", "GTU"]]        #validation 사용
    yvali = vali[["ELPS_MIN"]]

    eval_set = [(xvali, yvali)]

    test = new_df[["day", "hour", "mintue", "dayofweek", "GTU"]]            #예측데이터

    xgb_model = xgb.XGBRegressor(n_estimators=250, learning_rate=0.07, subsample=0.7,
                        colsample_bytree=0.7, max_depth=6, silent = 1, nthread = 4, min_child_weight = 4)


    xgb_model.fit(x, y, eval_set= eval_set, eval_metric="rmse", early_stopping_rounds=15)
    pred = xgb_model.predict(test)

    new_df['ELPS_MIN'] = pred

    return new_df

#-----------------------------------------------------ELPS_MSEC_MAX-------------------------------------------------------------------------

def ELPS_MAX(tr_1, new_df, vali):
    x = tr_1[["day", "hour", "mintue","dayofweek", 'GTU']]
    y = tr_1[["ELPS_MAX"]]

    xvali = vali[["day", "hour", "mintue", "dayofweek", "GTU"]]        #validation 사용
    yvali = vali[["ELPS_MAX"]]

    eval_set = [(xvali, yvali)]

    test = new_df[["day", "hour", "mintue", "dayofweek", "GTU"]]            #예측데이터

    xgb_model = xgb.XGBRegressor(n_estimators=250, learning_rate=0.07, subsample=0.7,
                        colsample_bytree=0.7, max_depth=6, silent = 1, nthread = 4, min_child_weight = 4)


    xgb_model.fit(x, y, eval_set= eval_set, eval_metric="rmse", early_stopping_rounds=15)
    pred = xgb_model.predict(test)

    new_df['ELPS_MAX'] = pred

    return new_df


#--------------------------------------------------------------TPS--------------------------------------------------------------------------------

df.day = df.day.astype('int')
df.hour = df.hour.astype('int')
df.mintue = df.mintue.astype('int')
new_df = pd.DataFrame()


#### tr_1은 7월 31일부터 8월 24일 
tr_1 = df.loc[df["TR_YMD_HMS"]<20220825000000]
tr_1 = df[['TPS', 'GTU', 'ELPS_AVG', 'ELPS_MIN', 'ELPS_MAX', 'Date', 'year', 'month', 'day', 'hour', 'mintue', 'dayofweek']]

            
            
            



#### comp은 8월 26일부터 8월 13일
comp = df.loc[df["TR_YMD_HMS"]>=20220825000000]
comp = comp[['TPS', 'GTU', 'ELPS_AVG', 'ELPS_MIN', 'ELPS_MAX', 'Date', 'year', 'month', 'day', 'hour', 'mintue', 'dayofweek']]

#----------------------------------------test dataset----------------------------------------------------------------------------------
test = pd.read_csv("./shooting_log (1).csv")
test= test.dropna()
test['Date'] = test.TR_YMD_HMS.apply(lambda x: str(x)[:14])
test['year'] = test.TR_YMD_HMS.apply(lambda x: str(x)[:4])
test['month'] = test.TR_YMD_HMS.apply(lambda x: str(x)[4:6])
test['day'] = test.TR_YMD_HMS.apply(lambda x: str(x)[6:8])
test['hour'] = test.TR_YMD_HMS.apply(lambda x: str(x)[8:10])
test['mintue'] = test.TR_YMD_HMS.apply(lambda x: str(x)[10:12])


test.Date = pd.to_datetime(test.Date.astype(str), format="%Y%m%d%H%M%S")

test["dayofweek"] = test.Date.dt.dayofweek


In [50]:
test

,TR_YMD_HMS,TPS,ELPS_AVG,ELPS_MIN,ELPS_MAX,Date,year,month,day,hour,mintue,dayofweek
0,2.022090e+13,476.42,0.175856,0.0,108.452386,2022-09-01 00:00:00,2022,09,01,00,00,3
1,2.022090e+13,325.64,0.217200,0.0,91.222438,2022-09-01 00:10:00,2022,09,01,00,10,3
2,2.022090e+13,264.84,0.220549,0.0,282.243435,2022-09-01 00:20:00,2022,09,01,00,20,3
3,2.022090e+13,262.51,0.185086,0.0,160.733486,2022-09-01 00:30:00,2022,09,01,00,30,3
4,2.022090e+13,227.51,0.215876,0.0,170.647935,2022-09-01 00:40:00,2022,09,01,00,40,3
...,...,...,...,...,...,...,...,...,...,...,...,...
111,2.022090e+13,1097.44,0.199853,0.0,492.334100,2022-09-01 18:30:00,2022,09,01,18,30,3
112,2.022090e+13,1008.28,0.204725,0.0,327.763873,2022-09-01 18:40:00,2022,09,01,18,40,3
113,2.022090e+13,968.31,0.191824,0.0,467.758472,2022-09-01 18:50:00,2022,09,01,18,50,3
114,2.022090e+13,980.09,0.178041,0.0,252.212044,2022-09-01 19:00:00,2022,09,01,19,00,3


In [51]:
comp

,TPS,GTU,ELPS_AVG,ELPS_MIN,ELPS_MAX,Date,year,month,day,hour,mintue,dayofweek
3600,470.655501,47.065550,0.201745,0.0,62.578475,2022-08-25 00:00:00,2022,08,25,0,0,3
3601,436.819396,43.681940,0.197355,0.0,53.737345,2022-08-25 00:10:00,2022,08,25,0,10,3
3602,416.546189,41.654619,0.192734,0.0,48.380999,2022-08-25 00:20:00,2022,08,25,0,20,3
3603,360.799750,36.079975,0.188169,0.0,47.190101,2022-08-25 00:30:00,2022,08,25,0,30,3
3604,322.810800,32.281080,0.183908,0.0,50.347604,2022-08-25 00:40:00,2022,08,25,0,40,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4603,667.448105,66.744811,0.227514,0.0,67.762486,2022-08-31 23:10:00,2022,08,31,23,10,2
4604,621.485579,62.148558,0.227302,0.0,59.417595,2022-08-31 23:20:00,2022,08,31,23,20,2
4605,626.314471,62.631447,0.226128,0.0,48.704498,2022-08-31 23:30:00,2022,08,31,23,30,2
4606,577.451892,57.745189,0.223949,0.0,36.550335,2022-08-31 23:40:00,2022,08,31,23,40,2


In [54]:
if __name__ == '__main__':
    new_df = GTU(tr_1, new_df, comp, test)
    new_df = ELPS_SUM(tr_1, new_df, comp)
    new_df = ELPS_MIN(tr_1, new_df, comp)
    new_df = ELPS_MAX(tr_1, new_df, comp)

    x = tr_1[["day", "hour", "mintue","dayofweek", 'GTU', "ELPS_AVG", "ELPS_MIN", "ELPS_MAX"]]
    y = tr_1[["TPS"]]

    xvali = comp[["day", "hour", "mintue","dayofweek", 'GTU', "ELPS_AVG", "ELPS_MIN", "ELPS_MAX"]]
    yvali = comp[["TPS"]]

    eval_set = [(xvali, yvali)]

    test = new_df[["day", "hour", "mintue", "dayofweek", "GTU", "ELPS_AVG", "ELPS_MIN", "ELPS_MAX"]]          

    xgb_model = xgb.XGBRegressor(n_estimators=950, learning_rate=0.07, subsample=0.7,
                        colsample_bytree=0.7, max_depth=6, silent = 1, nthread = 4, min_child_weight = 4)


    xgb_model.fit(x, y, eval_set= eval_set, eval_metric="rmse", early_stopping_rounds=15)

    pred = xgb_model.predict(test)

    new_df["TPS"] = pred

day          int32
hour         int32
mintue       int32
dayofweek    int64
dtype: object


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


ValueError: Experimental support for categorical data is not implemented for current tree method yet.

In [ ]:
print(test_train.dtypes)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import rc
rc('font', family='AppleGothic')


from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd



tr_1["TPS_MIN"] = tr_1["TPS"] * 0.33
tr_1["TPS_MAX"] = tr_1["TPS"] * 1.66

comp["TPS_MIN"] = comp["TPS"] * 0.33
comp["TPS_MAX"] = comp["TPS"] * 1.66

new_df["TPS_MIN"] = new_df["TPS"] * 0.33
new_df["TPS_MAX"] = new_df["TPS"] * 1.66


fig = plt.figure(figsize=(18,18)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

plt.xlabel("date_time") 
plt.ylabel('TPS')
plt.xticks(rotation=45)
ax.xaxis.set_major_locator(dates.MinuteLocator(interval = 10))



ax.plot(tr_1['Date'], tr_1["TPS"], marker='',label= '7/31 - 8/6 (train)', color = "g", linewidth = 2.5)
ax.plot(tr_1['Date'], tr_1["TPS_MIN"], color = "seagreen", linewidth = 0.4)
ax.plot(tr_1['Date'], tr_1["TPS_MAX"], color = "seagreen", linewidth = 0.4)
plt.fill_between(tr_1['Date'], tr_1["TPS_MIN"], tr_1["TPS_MAX"], color = "seagreen", alpha=0.15)


ax.plot(comp['Date'], comp["TPS"], marker='', label= "8/7 - 8/11 (validation)", color = "blue",linewidth = 2.5) 
ax.plot(comp['Date'], comp["TPS_MIN"], color = "cornflowerblue", linewidth = 0.4)
ax.plot(comp['Date'], comp["TPS_MAX"], color = "cornflowerblue", linewidth = 0.4)
plt.fill_between(comp['Date'], comp["TPS_MIN"], comp["TPS_MAX"], color = "cornflowerblue", alpha=0.15)


ax.plot(new_df['Date'], new_df["TPS"], marker='', label= "8/11 19시 - 8/13 19시", color="red", linewidth = 2.5) 
ax.plot(new_df['Date'], new_df["TPS_MIN"], color = "lightcoral", linewidth = 0.4)
ax.plot(new_df['Date'], new_df["TPS_MAX"], color = "lightcoral", linewidth = 0.4)
plt.fill_between(new_df['Date'], new_df["TPS_MIN"], new_df["TPS_MAX"], color = "lightcoral", alpha=0.15)

plt.legend()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import rc
rc('font', family='AppleGothic')

tr_1["TPS_MIN"] = tr_1["TPS"] * 0.33
tr_1["TPS_MAX"] = tr_1["TPS"] * 1.66

comp["TPS_MIN"] = comp["TPS"] * 0.33
comp["TPS_MAX"] = comp["TPS"] * 1.66

new_df["TPS_MIN"] = new_df["TPS"] * 0.33
new_df["TPS_MAX"] = new_df["TPS"] * 1.66


fig = plt.figure(figsize=(18,18)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

plt.xlabel("date_time") 
plt.ylabel('TPS')
plt.xticks(rotation=45)
ax.xaxis.set_major_locator(dates.MinuteLocator(interval = 10))

ax.plot(tr_1['Date'], tr_1["TPS"], marker='',label= '7/31 - 8/6 (train)', color = "g", linewidth = 2.5)
ax.plot(tr_1['Date'], tr_1["TPS_MIN"], color = "seagreen", linewidth = 0.4)
ax.plot(tr_1['Date'], tr_1["TPS_MAX"], color = "seagreen", linewidth = 0.4)
plt.fill_between(tr_1['Date'], tr_1["TPS_MIN"], tr_1["TPS_MAX"], color = "seagreen", alpha=0.15)


ax.plot(comp['Date'], comp["TPS"], marker='', label= "8/7 - 8/11 (validation)", color = "blue",linewidth = 2.5) 
ax.plot(comp['Date'], comp["TPS_MIN"], color = "cornflowerblue", linewidth = 0.4)
ax.plot(comp['Date'], comp["TPS_MAX"], color = "cornflowerblue", linewidth = 0.4)
plt.fill_between(comp['Date'], comp["TPS_MIN"], comp["TPS_MAX"], color = "cornflowerblue", alpha=0.15)


ax.plot(new_df['Date'], new_df["TPS"], marker='', label= "9월 1일", color="red", linewidth = 2.5) 
ax.plot(new_df['Date'], new_df["TPS_MIN"], color = "lightcoral", linewidth = 0.4)
ax.plot(new_df['Date'], new_df["TPS_MAX"], color = "lightcoral", linewidth = 0.4)
plt.fill_between(new_df['Date'], new_df["TPS_MIN"], new_df["TPS_MAX"], color = "lightcoral", alpha=0.15)

plt.legend()
plt.show()

#----------------------------------------------------------visualization 02------------------------------------------------------------------

tr_1["TPS_MIN"] = tr_1["TPS"] * 0.33
tr_1["TPS_MAX"] = tr_1["TPS"] * 1.66

comp["TPS_MIN"] = comp["TPS"] * 0.33
comp["TPS_MAX"] = comp["TPS"] * 1.66

new_df["TPS_MIN"] = new_df["TPS"] * 0.33
new_df["TPS_MAX"] = new_df["TPS"] * 1.66

tr_1 = tr_1.reset_index()
comp = comp.reset_index()
new_df = new_df.reset_index()

fig = plt.figure(figsize=(18,18)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

plt.xlabel("date_time") 
plt.ylabel('TPS')
plt.xticks(rotation=45)
#ax.xaxis.set_major_locator(dates.MintueLocator(interval = 10))

ax.plot(tr_1.index, tr_1["TPS"], marker='',label= '7/31 - 8/6 (train)', color = "g", linewidth = 2.5)
ax.plot(tr_1.index, tr_1["TPS_MIN"], color = "seagreen", linewidth = 0.4)
ax.plot(tr_1.index, tr_1["TPS_MAX"], color = "seagreen", linewidth = 0.4)
plt.fill_between(tr_1.index, tr_1["TPS_MIN"], tr_1["TPS_MAX"], color = "seagreen", alpha=0.15)


ax.plot(comp.index, comp["TPS"], marker='', label= "8/7 - 8/11 (validation)", color = "blue",linewidth = 2.5) 
ax.plot(comp.index, comp["TPS_MIN"], color = "cornflowerblue", linewidth = 0.4)
ax.plot(comp.index, comp["TPS_MAX"], color = "cornflowerblue", linewidth = 0.4)
plt.fill_between(comp.index, comp["TPS_MIN"], comp["TPS_MAX"], color = "cornflowerblue", alpha=0.15)


ax.plot(new_df.index, new_df["TPS"], marker='', label= "8/11 19시 - 8/13 19시", color="red", linewidth = 2.5) 
ax.plot(new_df.index, new_df["TPS_MIN"], color = "lightcoral", linewidth = 0.4)
ax.plot(new_df.index, new_df["TPS_MAX"], color = "lightcoral", linewidth = 0.4)
plt.fill_between(new_df.index, new_df["TPS_MIN"], new_df["TPS_MAX"], color = "lightcoral", alpha=0.15)

plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import rc




rc('font', family='AppleGothic')

tr_1["TPS_MIN"] = tr_1["TPS"] * 0.33
tr_1["TPS_MAX"] = tr_1["TPS"] * 1.66

comp["TPS_MIN"] = comp["TPS"] * 0.33
comp["TPS_MAX"] = comp["TPS"] * 1.66

new_df["TPS_MIN"] = new_df["TPS"] * 0.33
new_df["TPS_MAX"] = new_df["TPS"] * 1.66


fig = plt.figure(figsize=(18,18)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

plt.xlabel("date_time") 
plt.ylabel('TPS')
plt.xticks(rotation=45)
ax.xaxis.set_major_locator(dates.MinuteLocator(interval = 10))

ax.plot(new_df['Date'], new_df["TPS"], marker='', label= "8/11 19시 - 8/13 19시", color="red", linewidth = 2.5) 
ax.plot(new_df['Date'], new_df["TPS_MIN"], color = "lightcoral", linewidth = 0.4)
ax.plot(new_df['Date'], new_df["TPS_MAX"], color = "lightcoral", linewidth = 0.4)
plt.fill_between(new_df['Date'], new_df["TPS_MIN"], new_df["TPS_MAX"], color = "lightcoral", alpha=0.15)


In [ ]:
import random

out_index = []
out_value = []

for i in range(1,10):
    out_index.append(random.randrange(0, len(new_df)-1))
    out_value.append(random.uniform(0, 5))

out_df = pd.DataFrame(out_value, index=out_index, columns = ['TPS'])
out_df = out_df.sort_index(ascending=True)



start_date = pd.to_datetime('2022-08-12 00:00:00') ## 시작 날짜
end_date = pd.to_datetime('2022-08-14 00:00:00') ## 마지막 날짜
dates = pd.date_range(start_date,end_date,freq='10min') ## 일단위로 생성

date_value=pd.DataFrame(dates)

date_value=date_value.rename(columns={0:"date"})

err = []
for i in out_df.index:
    if new_df.TPS_MIN[i] < out_df.TPS[i] < new_df.TPS_MAX[i]:
        err.append(0)
    else:
        err.append(1)
        
out_df=pd.merge(date_value,out_df,left_index=True, right_index=True)
        


out_df["error"] = err

In [ ]:

err = []
for i in out_df.index:
    if new_df.TPS_MIN[i] < out_df.TPS[i] < new_df.TPS_MAX[i]:
        err.append(0)
    else:
        err.append(1)
   
        


out_df["error"] = err



fal = out_df[out_df.error == 1]
nor = out_df[out_df.error == 0]

In [ ]:



fig = plt.figure(figsize=(18,18)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

plt.xlabel("date_time") 
plt.ylabel('TPS')
plt.xticks(rotation=45)
#ax.xaxis.set_major_locator(dates.MintueLocator(interval = 10))



ax.plot(new_df["Date"], new_df["TPS"], marker='', label= "8/11 19시 - 8/13 19시", color="red", linewidth = 2.5) 
ax.plot(new_df["Date"], new_df["TPS_MIN"], color = "lightcoral", linewidth = 0.4)
ax.plot(new_df["Date"], new_df["TPS_MAX"], color = "lightcoral", linewidth = 0.4)
plt.fill_between(new_df["Date"], new_df["TPS_MIN"], new_df["TPS_MAX"], color = "lightcoral", alpha=0.15)


ax.scatter(nor["date"], nor.TPS,  marker = 'o', color = "yellow", s = 50)
ax.scatter(fal["date"], fal.TPS,  marker = '*', color = "black", s = 50)
plt.legend()
plt.show()

In [ ]:

def init():
    ax.plot(new_df["Date"], new_df["TPS"], marker='', label= "8/11 19시 - 8/13 19시", color="red", linewidth = 2.5) 
    ax.plot(new_df["Date"], new_df["TPS_MIN"], color = "lightcoral", linewidth = 0.4)
    ax.plot(new_df["Date"], new_df["TPS_MAX"], color = "lightcoral", linewidth = 0.4)
    plt.fill_between(new_df["Date"], new_df["TPS_MIN"], new_df["TPS_MAX"], color = "lightcoral", alpha=0.15)

    return ax



figure, ax = plt.subplots(figsize=(10,10))


def func_animate(num):
    x = list(out_df["date"])[num]
    y = list(out_df["TPS"])[num]
    
    
    if list(out_df[0])[num] == 1:
        ax.scatter(x ,y,  marker = '*', color = "blue", s = 60)
        
    else:
        ax.scatter(x ,y,  marker = 'o', color = "red", s = 50)
 



anim = FuncAnimation(figure,
                    func_animate,
                    init_func=init,
                    frames=len(out_df["date"]),
                    interval=0.0025)

anim.save('animatio242n.gif', fps=2)

plt.show()